<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/paths_Tapis.ipynb" 
    target="_blank"
    >
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# Tapis Paths  📒
***How DesignSafe File Storage and Tapis Work Together***

by Silvia Mazzoni, DesignSafe, 2025

Tapis powers file access and job submission on DesignSafe. It provides a consistent interface to interact with **multiple storage systems** and **compute environments**, making it easier to manage data before, during, and after simulation workflows.

With Tapis, you can:

* **List, upload, download, move, and delete files** across different storage systems
* **Stage input files** (e.g., move from long-term storage to a compute node)
* **Collect outputs automatically** and return them to Corral (MyData)
* Use the same scripting or automation tools across storage locations

Tapis acts as the **glue** between DesignSafe’s storage and compute environments, streamlining data movement and improving reproducibility.

---


In [1]:
# ## DesignSafe File Systems & Tapis

# Tapis connects to several distinct file storage systems, each with a unique role. You interact with them using **URI-style paths** like:

# ```
# tapis://<base-path>/<relative-path>
# ```
# When working in Jupyter Hub


# Although Tapis paths look like web URLs, they reference the storage systems storing the files. While some storage systems have unique Tapis paths, many of them are user, system, and/or project dependent.



# Here’s a reference table for Tapis paths:

# | Storage Type        | Example Tapis Path                            | Notes                                  | Exceptions
# | ------------------- | --------------------------------------------- | -------------------------------------- |------------
# | **MyData**          | tapis://designsafe.storage.default/jdoe/      | Your personal storage                  | 
# | **Community**       | tapis://designsafe.storage.community/         | Public files from the community        | 
# | **Work**            | tapis://cloud.data/work/05072/jdoe/stampede3/ | Shared group allocation; used for jobs | user-specific
# | **MyProjects**      | tapis://project-61bab56…                      | Linked to specific DesignSafe projects | project-specific
# | **NHERI Published** | tapis://designsafe.storage.published/PRJ-1628 | Archived datasets                      |
# | **NEES Published**  | tapis://nees.public/NEES-2                    | Legacy NEES content                    |
# | **Scratch**         | Not available through Tapis explicitly        | Temporary compute storage              | not available
# | **Home**            | Not available through Tapis                   | Home directories on HPC systems        | not available

# ⚠️ Each URI must include the correct **system name** and **path** (which often includes your username, group ID, or project ID). There is **no built-in Tapis command** to automatically discover your base paths — but you can find them manually (see below).


In [2]:
import json

In [3]:
# Local Utilities Library
# you can remove the logic associated with the local path
import sys,os
relativePath = '../OpsUtils'
if os.path.exists(relativePath):
    print("Using local utilities library")
    PathOpsUtils = os.path.expanduser(relativePath)
else:
    print('using communitydata')
    PathOpsUtils = os.path.expanduser('~/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils

Using local utilities library


---
## Connect to Tapis

In [4]:
t=OpsUtils.connect_tapis()

 -- Checking Tapis token --
 Token loaded from file. Token is still valid!
 Token expires at: 2025-08-23T01:42:21+00:00
 Token expires in: 3:20:52.782796
-- LOG IN SUCCESSFUL! --


---
## Configure App

In [5]:
app_id = 'stampede3-credential'
app_version = "1.0.0"

#### List the app schema

In [6]:
appMetaData = t.apps.getAppLatestVersion(appId=app_id)
OpsUtils.display_tapis_app_schema(appMetaData)

########################################
########### TAPIS-APP SCHEMA ###########
########################################
######## appID: stampede3-credential
######## version: 1.0.0
########################################
{
  sharedAppCtx: "wma_prtl"
  isPublic: True
  tenant: "designsafe"
  id: "stampede3-credential"
  version: "1.0.0"
  description: "Credential application for Stampede3"
  owner: "wma_prtl"
  enabled: True
  versionEnabled: True
  locked: False
  runtime: "SINGULARITY"
  runtimeVersion: None
  containerImage: "docker://alpine:latest"
  jobType: "FORK"
  maxJobs: 2147483647
  maxJobsPerUser: 2147483647
  strictFileInputs: True
  uuid: "cfb2d592-eb94-4f88-8ecc-223e79b974e6"
  deleted: False
  created: "2024-08-13T16:57:47.789293Z"
  updated: "2025-02-26T21:17:26.144467Z"
  sharedWithUsers: []
  runtimeOptions: ["SINGULARITY_RUN"]
  tags: ["portalName: DesignSafe"]
  jobAttributes: {
    description: ""
    dynamicExecSystem: False
    execSystemConstraints: None
   

---
## Submit a Job

You can now submit a job using this app. You can use the Tapis CLI, Tapipy, or a web form.

We are using TapiPy directly from this notebook. We will not specify a version so that the latest is used by default in the description.

In [7]:
job_def = {
    "name": "print-work",
    "appId": app_id,
    "appVersion":app_version,
}

In [8]:
resp = t.jobs.submitJob(**job_def)

In [9]:
work_dir = resp.archiveSystemDir
print('work_dir',work_dir)

work_dir /work2/05072/silvia/stampede3/tapis-jobs-archive/2025-08-22Z/print-work-a3657991-915d-4690-aa21-7513d107238c-007


In [10]:
def get_user_work_path(t,systemID='stampede3'):
    system_id = systemID.lower()
    yesSystems = ['stampede3','ls6','frontera']
    latest = t.apps.getAppLatestVersion(appId=app_id)
    appVersion = latest.version
    if system_id  in yesSystems:
        job_description = {'name':'getWork','appId':f'{system_id}-credential','appVersion':appVersion}
    else:
        print(f'{systemID} does not exist! select from: {yesSystems}')
        return -1
    submitted_job = t.jobs.submitJob(**job_description)
    archiveSystemDir = submitted_job.archiveSystemDir
    workDir = archiveSystemDir.split(system_id)[0] + system_id
    return workDir
    

In [11]:
OpsUtils.get_user_work_tapis_uri(t,'stampede3')

'tapis://cloud.data/work2/05072/silvia/stampede3'

In [12]:
get_tapis_work_path(t,'stampede3')

NameError: name 'get_tapis_work_path' is not defined

In [ ]:
aa = '/hello/'
aa.rstrip('/')
